In [2]:
import pandas as pd

# Ruta del archivo CSV exportado desde StrategyQuant
file_path = '/mnt/c/Users/Administrador/Downloads/DatabankExport.csv'

# Cargar el CSV (con delimitador ";")
df = pd.read_csv(file_path, delimiter=';', encoding='utf-8')

# Función para procesar indicadores
def process_indicators(column_name):
    # Separar múltiples indicadores y "explotarlos" en filas individuales
    expanded = df.assign(**{column_name: df[column_name].str.split(',')}).explode(column_name)

    # Agrupar por nombre de indicador y calcular promedios
    grouped = expanded.groupby(column_name).agg({
        'Edge Ratio': 'mean',
        'Ret/DD Ratio': 'mean',
        'Profit factor': 'mean',
        column_name: 'count'  # cuántas veces aparece
    }).rename(columns={
        'Edge Ratio': 'Avg_EdgeRatio',
        'Ret/DD Ratio': 'Avg_RetDD',
        'Profit factor': 'Avg_ProfitFactor',
        column_name: 'Count'
    })

    # Calcular score ponderado
    grouped['Score'] = (
        grouped['Avg_EdgeRatio'] * 20 +
        grouped['Avg_RetDD'] * 15 +
        grouped['Avg_ProfitFactor'] * 15 +
        grouped['Count'] * 50
    )

    # Seleccionar los 15 mejores
    return grouped.sort_values(by='Score', ascending=False).head(15)

# Procesar ambos tipos de indicadores
top_entry_indicators = process_indicators('Entry indicators')
top_price_indicators = process_indicators('Price indicators')

# Preparar resultados para mostrar
entry = top_entry_indicators.reset_index()
price = top_price_indicators.reset_index()

# Rellenar si hay diferencias de longitud
max_len = max(len(entry), len(price))
entry = entry.reindex(range(max_len))
price = price.reindex(range(max_len))

# Combinar en DataFrame final
result = pd.DataFrame({
    'Top Entry Indicators': entry['Entry indicators'],
    'Entry Score': entry['Score'],
    'Top Price Indicators': price['Price indicators'],
    'Price Score': price['Score']
})
# Mostrar los resultados ordenados
display(result)

,Top Entry Indicators,Entry Score,Top Price Indicators,Price Score
0,LinearRegression,220.725,HeikenAshiClose,187.425
1,CloseM,183.150,BBWidthRatio,183.150
2,High,179.175,Open,183.150
3,Low,179.175,SmallestRange,169.175
4,GannHiLo,169.175,TrueRange,157.075
5,SessionHigh,158.300,KeltnerChannel,131.800
6,SessionLow,158.300,Fibo,115.300
7,SessionOpen,143.050,CloseM,106.550
8,KeltnerChannel,131.800,HighM,98.850
9,BarHour,116.150,NaN,NaN
